In [1]:
#Import Relevant Libraries
import random
import numpy as np
import cvxpy as cp

from qiskit import QuantumCircuit, execute, Aer, IBMQ, BasicAer
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector

from toqito.state_metrics import fidelity
from toqito.channels import partial_trace as pt
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace

In [2]:
'''
Defines parameters that specify the purifications of the three states.
'''
unitary_param_1 = [1.43122857, 0.69368875, 2.62909511, 0.56228069, 0.57376119, 0.65572289, 4.35150067, 0.4100615]
unitary_param_2 = [1.5368315, 3.21128373, 5.9793786, 0.22765385, 1.29319139, 3.70786542, 3.5394303, 5.14181531]
unitary_param_3 = [1.05605604, 1.9644681, 1.21408169, 5.93323246, 5.8882428, 0.79039103, 3.54426315, 0.23689953]

'''
Defines the probabilities of the individual states
'''   
[p_1, p_2, p_3] = [0.16279796495164395, 0.29323674476659134, 0.5439652902817648]

In [3]:
'''
Create purification of state 1
'''
size = 2
qc1 = QuantumCircuit(size)
for j in range(2):
    for i in range(size):
        qc1.rx(unitary_param_1[j*2*size + i], i)
        qc1.ry(unitary_param_1[j*2*size + i + size], i)  

    for i in range(size-1):
        qc1.cx(i, i+1)

qc1.draw()

┌────────────┐ ┌────────────┐     ┌─────────────┐ ┌────────────┐     
q_0: ─┤ Rx(1.4312) ├─┤ Ry(2.6291) ├──■──┤ Rx(0.57376) ├─┤ Ry(4.3515) ├──■──
     ┌┴────────────┤┌┴────────────┤┌─┴─┐├─────────────┤┌┴────────────┤┌─┴─┐
q_1: ┤ Rx(0.69369) ├┤ Ry(0.56228) ├┤ X ├┤ Rx(0.65572) ├┤ Ry(0.41006) ├┤ X ├
     └─────────────┘└─────────────┘└───┘└─────────────┘└─────────────┘└───┘

In [4]:
'''
Create purification of state 2
'''
size = 2
qc2 = QuantumCircuit(size)
for j in range(2):
    for i in range(size):
        qc2.rx(unitary_param_2[j*2*size + i], i)
        qc2.ry(unitary_param_2[j*2*size + i + size], i)

    for i in range(size-1):
        qc2.cx(i, i+1)

qc2.draw()

┌────────────┐ ┌────────────┐     ┌────────────┐┌────────────┐     
q_0: ┤ Rx(1.5368) ├─┤ Ry(5.9794) ├──■──┤ Rx(1.2932) ├┤ Ry(3.5394) ├──■──
     ├────────────┤┌┴────────────┤┌─┴─┐├────────────┤├────────────┤┌─┴─┐
q_1: ┤ Rx(3.2113) ├┤ Ry(0.22765) ├┤ X ├┤ Rx(3.7079) ├┤ Ry(5.1418) ├┤ X ├
     └────────────┘└─────────────┘└───┘└────────────┘└────────────┘└───┘

In [5]:
'''
Create purification of state 3
'''
size = 2
qc3 = QuantumCircuit(size)
for j in range(2):
    for i in range(size):
        qc3.rx(unitary_param_3[j*2*size + i], i)
        qc3.ry(unitary_param_3[j*2*size + i + size], i)

    for i in range(size-1):
        qc3.cx(i, i+1)

qc3.draw()

┌────────────┐┌────────────┐      ┌────────────┐┌────────────┐     
q_0: ┤ Rx(1.0561) ├┤ Ry(1.2141) ├──■───┤ Rx(5.8882) ├┤ Ry(3.5443) ├──■──
     ├────────────┤├────────────┤┌─┴─┐┌┴────────────┤├────────────┤┌─┴─┐
q_1: ┤ Rx(1.9645) ├┤ Ry(5.9332) ├┤ X ├┤ Rx(0.79039) ├┤ Ry(0.2369) ├┤ X ├
     └────────────┘└────────────┘└───┘└─────────────┘└────────────┘└───┘

In [6]:
'''
Creates the density matrix rho_XB which encodes the overall state.
'''
psi_1 = np.array(DensityMatrix.from_instruction(qc1.reverse_bits()))
psi_2 = np.array(DensityMatrix.from_instruction(qc2.reverse_bits()))
psi_3 = np.array(DensityMatrix.from_instruction(qc3.reverse_bits()))

rho_1 = pt(psi_1, [1], [2, 2])
rho_2 = pt(psi_2, [1], [2, 2])
rho_3 = pt(psi_3, [1], [2, 2])

rho_1_XB = np.kron(np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]]), rho_1)
rho_2_XB = np.kron(np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]]), rho_2)
rho_3_XB = np.kron(np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]]), rho_3)

rho_XB = p_1*rho_1_XB + p_2*rho_2_XB + p_3*rho_3_XB

In [7]:
#Primal Formulation
M_dim = rho_XB.shape
M = cp.Variable(M_dim, complex=True)
c = [pt(M, [1], [3, 2]) == np.identity(2), M>>0]
obj = cp.Maximize(cp.real(cp.trace(M@rho_XB)))
problem = cp.Problem(obj, constraints = c)
problem.solve(verbose=False)
print("Optimal Probability : " + str(problem.value))

Optimal Probability : 0.6723349065179771


In [8]:
#Dual Formulation
Y_dim = (2, 2)
Y = cp.Variable(Y_dim, complex=True)
c = [cp.kron(np.identity(3), Y) >>rho_XB , Y>>0]
obj = cp.Minimize(cp.real(cp.trace(Y)))
problem = cp.Problem(obj, constraints = c)
problem.solve(verbose=False)
print("Optimal Probability : " + str(problem.value))

WARN: A->p (column pointers) not strictly increasing, column 4 empty
WARN: A->p (column pointers) not strictly increasing, column 7 empty
Optimal Probability : 0.6723353740608686
